# Getting starting fine-tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [ ]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

Install all required dependencies:

In [ ]:
%pip install -r requirements.txt

## Model download

In [ ]:
!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

In [ ]:
!tar -xf mistral-7B-v0.3.tar -C ./mistral_models

In [ ]:
# Alternatively, you can download the model from Hugging Face

# !pip install huggingface_hub
# from huggingface_hub import snapshot_download
# from pathlib import Path

# mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
# mistral_models_path.mkdir(parents=True, exist_ok=True)

# snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

#! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
#! rm -r /root/mistral_models/7B-v0.3

In [ ]:
!ls mistral_models

## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [ ]:
# navigate to this data directory
# root_path = "/home/linx/smis/codes/ollama/openkh/content"
%cd data

In [ ]:
import pandas as pd
# df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')
df = pd.read_parquet('data/find-tune/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

In [ ]:
df.head()

In [ ]:
import string, random
len(df['prompt_id'][5])

In [ ]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [ ]:
# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [ ]:
import pandas as pd

df = pd.read_parquet('data/function-calling/locutusque-function-calling-chatml.parquet')

In [ ]:
df['function_description'][11].strip()

In [ ]:
import json, random, string

datas_call = []
# file_path = 'ultrachat_chunk_train.jsonl'
for i in range(len(df)):
    message = df['conversations'][i]
    system_prompt = df['system_message'][i]
    tools = df['function_description'][i]
    if len(tools)>0 and isinstance(tools, list):
        tools = [{"type": "function","function": i} for i in tools]
    elif tools.strip() != "":
        tools = [{"type": "function","function": tools}]
    
    messagesort = []
    mess = {}
    for i in message:
        format_message = {'content': '',"role": 'user'}
        if i['from'] == 'human':
            prompt = i['value']
            role = 'user'
        elif i['from'] == 'gpt':
            role = 'assistant'
        elif i['from'] == 'function-call':
            role = 'tool'
        # elif i['from'] == 'function-response':
        #     role = 'system'
        else:
            role = i['from']
        
        prompt_id = ''.join(random.choices(string.ascii_letters + string.digits, k=64))

        format_message['content'] = i['value']
        format_message['role'] = role
        mess[role]=format_message
        # if role not in ['system', 'user', 'assistant', 'tool']:
        #     continue
        # else:
        #     mess[role]=format_message
        
    for key in ['system', 'user', 'assistant', 'tool']:
        if key in mess:
            messagesort.append(mess[key])
    data = {"prompt": prompt, "prompt_id": prompt_id, "messages": messagesort}
    if isinstance(tools, list):
        data['tools'] = tools
    datas_call.append(json.loads(json.dumps(data)))

In [ ]:
# datas_call
# dfs = pd.DataFrame(datas_call)

In [ ]:
len(datas_call[:30])

In [ ]:
dfs = pd.DataFrame(datas_call)
# split data into training and evaluation
df_train=dfs.sample(frac=0.95,random_state=200)
df_eval=dfs.drop(df_train.index)

In [ ]:
# from IPython.display import display, Markdown, Latex
# display(Markdown(df_train['tools']))

In [ ]:
# save data into .jsonl files
%rm -rf data/function-calling/ultrachat*
df_train.to_json("data/function-calling/ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("data/function-calling/ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [ ]:
# path = f"{root_path}/data"
# %cd ..
!ls data/function-calling/

In [ ]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that doesn't have the right format:

!python -m utils.reformat_data data/function-calling/ultrachat_chunk_train.jsonl

In [ ]:
# eval data looks all good
!python -m utils.reformat_data data/function-calling/ultrachat_chunk_eval.jsonl

In [ ]:
from tqdm import tqdm
from IPython.display import Markdown, display
import json

with open('data/function-calling/ultrachat_chunk_train.jsonl', "r", encoding="utf-8") as f:
    lines = f.readlines()
    for idx, line in tqdm(enumerate(lines), total=len(lines)):
        data = json.loads(line)
        # print(data)
        # continue
        # break
        if idx == 60:
            for i, mess in enumerate(data['messages']):
                print(i,mess['role'],mess['content'])

In [ ]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/fc-mistral7b.yaml

## Start training

In [ ]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
os.environ

In [ ]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence lentgh of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 8192
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [ ]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

In [ ]:
# from torch.cuda.amp import autocast, GradScaler

# scaler = GradScaler()

# for data, target in train_loader:
#     optimizer.zero_grad()
#     with autocast():
#         output = model(data)
#         loss = criterion(output, target)
#     scaler.scale(loss).backward()
#     scaler.step(optimizer)
#     scaler.update()

In [ ]:
# start training
%rm -rf test_ultra
# !torchrun --nproc-per-node 1 -m train example.yaml
!torchrun --nproc-per-node 1 -m train example/fc-mistral7b.yaml
# !torchrun --nproc-per-node 1 -m train example/test7b.yaml

## Inference

In [ ]:
%pip install mistral_inference

In [ ]:
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("mistral_models")  # change to extracted model dir
model.load_lora("mistral_models/consolidated.safetensors")
# model.load_lora("test_ultra/checkpoints/checkpoint_000100/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)